# FREQUENZA

Frequenza calcolata con wordfreq.word_frequency() e normalizzata rispetto alla frequenza media.

## Utilities

In [33]:
import nltk
from nltk.corpus import wordnet
import pandas as pd
from wordfreq import word_frequency, zipf_frequency

In [34]:
import sys
sys.path.insert(0, '../../')
from commons_init import SYNSET_DA_EVITARE, SUPPORTED_POS

## Calcolo Frequenza

- Estraggo frequenza con wordfreq (https://pypi.org/project/wordfreq/)

    - **word_frequency()** : frequenza reale di una parola all'interno di un corpus di testo specifico
    - **zipf_frequency()** : frequenza prevista di una parola secondo la legge di Zipf, stima della frequenza di una parola sulla base di una tendenza empirica generale osservata nei corpus di testo naturali

- Nel calcolo del basicness score ML *i risultati migliori si hanno con word_frequency() normalizzata rispetto alla frequenza media*; tale versione è quella qui applicata.
    - È stato mantenuto anche il calcolo con *zipf_frequency()* per eventuali confronti.

- Una frequenza più elevata suggerisce che la parola è basic.

### Frequenza di una parola specifica
- Non tengo in considerazione tutti i lemmi, solo quello base!!
- Parole composte: non cambia se uso delimitatore " " o "-"; frequenza = 0 se uso delimitatore "_" o altro.

In [35]:
def get_synset_freq(synset, method):
    method_functions = {
        "wordfreq": word_frequency,
        "zipf": zipf_frequency
    }
    
    synset_name = synset.lemmas()[0].name().replace('_', ' ')

    if method not in method_functions:
        raise ValueError(f"Metodo sconosciuto: {method}")
    synset_freq = method_functions[method](synset_name, 'en')
    
    return synset_freq

#### word_frequency()

In [36]:
synset = wordnet.synset('physical_entity.n.01')
synset_wordfreq = get_synset_freq(synset, "wordfreq")
print(f"Frequenza di '{synset}': {synset_wordfreq}")

Frequenza di 'Synset('physical_entity.n.01')': 1.14e-05


#### zipf_frequency()

In [37]:
synset = wordnet.synset('physical_entity.n.01')
synset_zipf_freq = get_synset_freq(synset, "zipf")
print(f"Frequenza di '{synset}': {synset_zipf_freq}")

Frequenza di 'Synset('physical_entity.n.01')': 4.06


### Calcolo frequenza di tutte le parole

In [38]:
def compute_all_synsets_freqs(method):
    synset_freq_dict = {}
    
    for pos in SUPPORTED_POS:
        synsets = wordnet.all_synsets(pos)
        
        for synset in synsets:
            if synset in SYNSET_DA_EVITARE or synset.pos() == 's':
                continue
            
            try:
                synset_freq = get_synset_freq(synset, method)
                synset_freq_dict[synset] = synset_freq
            
            except Exception as e:
                print(f"Errore durante il calcolo della frequenza per il synset {synset}: {e}")
    
    return synset_freq_dict

#### word_frequency()

In [39]:
synset_wordfreq_dict = compute_all_synsets_freqs("wordfreq")

In [40]:
# synset con frequenza = 0
synsets_wordfreq_zero = [syn for syn, freq in synset_wordfreq_dict.items() if freq == 0]
print(synsets_wordfreq_zero)
print(f'Numero di synset con frequenza = 0: {len(synsets_wordfreq_zero)}')

[Synset('heterotroph.n.01'), Synset('biont.n.01'), Synset('decampment.n.02'), Synset('abscondment.n.01'), Synset('deviationism.n.01'), Synset('nonaccomplishment.n.01'), Synset('footfault.n.01'), Synset('ballup.n.01'), Synset('viatication.n.01'), Synset('preoccupancy.n.02'), Synset('overcapitalization.n.01'), Synset('extemporization.n.01'), Synset('beanball.n.01'), Synset('countershot.n.01'), Synset('ballottement.n.01'), Synset('doweling.n.01'), Synset('fomentation.n.03'), Synset('blandishment.n.02'), Synset('shtik.n.04'), Synset('metrification.n.02'), Synset('supersedure.n.01'), Synset('whitelash.n.01'), Synset('nonacceptance.n.01'), Synset('anathematization.n.01'), Synset('conge.n.03'), Synset('mariticide.n.01'), Synset('uxoricide.n.02'), Synset('cardiospasm.n.01'), Synset('laryngismus.n.01'), Synset('aborticide.n.02'), Synset('recission.n.01'), Synset('vitiation.n.01'), Synset('hypnogenesis.n.01'), Synset('tenderization.n.01'), Synset('disinfestation.n.01'), Synset('hairweaving.n.01'

In [41]:
primi_10_valori = {k: synset_wordfreq_dict[k] for k in list(synset_wordfreq_dict)[:10]}
print(primi_10_valori)

{Synset('entity.n.01'): 1.32e-05, Synset('physical_entity.n.01'): 1.14e-05, Synset('abstraction.n.06'): 2e-06, Synset('thing.n.12'): 0.000525, Synset('object.n.01'): 3.63e-05, Synset('whole.n.02'): 0.000288, Synset('congener.n.03'): 4.68e-08, Synset('living_thing.n.01'): 0.000145, Synset('organism.n.01'): 3.89e-06, Synset('benthos.n.02'): 5.13e-08}


In [42]:
max_freq_word = max(synset_wordfreq_dict, key=synset_wordfreq_dict.get)
value_max_freq = synset_wordfreq_dict[max_freq_word]
print(f"Parola con la frequenza massima: {max_freq_word}")
print(f"Frequenza della parola con la frequenza massima: {value_max_freq}")

Parola con la frequenza massima: Synset('a.n.07')
Frequenza della parola con la frequenza massima: 0.0229


#### zipf_frequency()

In [43]:
synset_zipf_freq_dict = compute_all_synsets_freqs("zipf")

In [44]:
# synset con frequenza = 0
synsets_zipf_freq_zero = [syn for syn, freq in synset_zipf_freq_dict.items() if freq == 0]
print(synsets_zipf_freq_zero)
print(f'Numero di synset con frequenza = 0: {len(synsets_zipf_freq_zero)}')

[Synset('heterotroph.n.01'), Synset('biont.n.01'), Synset('decampment.n.02'), Synset('abscondment.n.01'), Synset('deviationism.n.01'), Synset('nonaccomplishment.n.01'), Synset('footfault.n.01'), Synset('ballup.n.01'), Synset('viatication.n.01'), Synset('preoccupancy.n.02'), Synset('overcapitalization.n.01'), Synset('extemporization.n.01'), Synset('beanball.n.01'), Synset('countershot.n.01'), Synset('ballottement.n.01'), Synset('doweling.n.01'), Synset('fomentation.n.03'), Synset('blandishment.n.02'), Synset('shtik.n.04'), Synset('metrification.n.02'), Synset('supersedure.n.01'), Synset('whitelash.n.01'), Synset('nonacceptance.n.01'), Synset('anathematization.n.01'), Synset('conge.n.03'), Synset('mariticide.n.01'), Synset('uxoricide.n.02'), Synset('cardiospasm.n.01'), Synset('laryngismus.n.01'), Synset('aborticide.n.02'), Synset('recission.n.01'), Synset('vitiation.n.01'), Synset('hypnogenesis.n.01'), Synset('tenderization.n.01'), Synset('disinfestation.n.01'), Synset('hairweaving.n.01'

In [45]:
primi_10_valori = {k: synset_zipf_freq_dict[k] for k in list(synset_zipf_freq_dict)[:10]}
print(primi_10_valori)

{Synset('entity.n.01'): 4.12, Synset('physical_entity.n.01'): 4.06, Synset('abstraction.n.06'): 3.3, Synset('thing.n.12'): 5.72, Synset('object.n.01'): 4.56, Synset('whole.n.02'): 5.46, Synset('congener.n.03'): 1.67, Synset('living_thing.n.01'): 5.16, Synset('organism.n.01'): 3.59, Synset('benthos.n.02'): 1.71}


In [46]:
max_freq_word = max(synset_zipf_freq_dict, key=synset_zipf_freq_dict.get)
value_max_freq = synset_zipf_freq_dict[max_freq_word]
print(f"Parola con la frequenza massima: {max_freq_word}")
print(f"Frequenza della parola con la frequenza massima: {value_max_freq}")

Parola con la frequenza massima: Synset('a.n.07')
Frequenza della parola con la frequenza massima: 7.36


### Normalizzazione della frequenza delle parole

- Se normalizzo rispetto alla media, i synset con frequenza maggiore della media avranno valore normalizzato >1. Per normalizzare anche questi li metto =1 e per gli altri tengo il valore normalizzato.

#### word_frequency()

In [47]:
# normalizzo rispetto alla frequenza media. quelli sopra la frequenza media li metto = 1.
# risultati migliori per basicness score ML usando word_frequency()

def normalized_wordfreq_score(dict_syn_freq):
    norm_synset_freq_dict = {}
    
    average_frequency = sum(dict_syn_freq.values()) / len(dict_syn_freq)
    for synset, freq in dict_syn_freq.items():
        if freq == 0:
            norm_synset_freq_dict[synset] = 0
        else:
            norm_freq_score = freq / average_frequency
            norm_freq_score = 1 if norm_freq_score >= 1 else norm_freq_score
            norm_synset_freq_dict[synset] = norm_freq_score
    
    return norm_synset_freq_dict, average_frequency

In [48]:
norm_synset_wordfreq_dict, wordfreq_average_frequency = normalized_wordfreq_score(synset_wordfreq_dict)

In [49]:
primi_10_valori = {k: norm_synset_wordfreq_dict[k] for k in list(norm_synset_wordfreq_dict)[:10]}
print(primi_10_valori)

{Synset('entity.n.01'): 0.7657463576738481, Synset('physical_entity.n.01'): 0.6613263998092324, Synset('abstraction.n.06'): 0.11602217540512849, Synset('thing.n.12'): 1, Synset('object.n.01'): 1, Synset('whole.n.02'): 1, Synset('congener.n.03'): 0.002714918904480007, Synset('living_thing.n.01'): 1, Synset('organism.n.01'): 0.22566313116297493, Synset('benthos.n.02'): 0.002975968799141546}


#### zipf_frequency()

In [50]:
# # normalizzo rispetto al massimo e al minimo
# # risultati pessimi su basicness score ML
# per zipf è meglio usare questa normalizzazione

def normalized_freq_score_zipf(dict_syn_freq):
    norm_synset_freq_dict = {}
    
    average_frequency = sum(dict_syn_freq.values()) / len(dict_syn_freq)
    min_freq = min(dict_syn_freq.values())
    max_freq = max(dict_syn_freq.values())
    
    for synset, freq in dict_syn_freq.items():
        norm_freq = (freq - min_freq) / (max_freq - min_freq)
        norm_synset_freq_dict[synset] = norm_freq
    
    return norm_synset_freq_dict, average_frequency

In [51]:
zipf_norm_synset_freq_dict, zipf_average_frequency = normalized_freq_score_zipf(synset_zipf_freq_dict)

In [52]:
primi_10_valori = {k: zipf_norm_synset_freq_dict[k] for k in list(zipf_norm_synset_freq_dict)[:10]}
print(primi_10_valori)

{Synset('entity.n.01'): 0.5597826086956522, Synset('physical_entity.n.01'): 0.5516304347826086, Synset('abstraction.n.06'): 0.44836956521739124, Synset('thing.n.12'): 0.7771739130434782, Synset('object.n.01'): 0.6195652173913042, Synset('whole.n.02'): 0.7418478260869564, Synset('congener.n.03'): 0.22690217391304346, Synset('living_thing.n.01'): 0.7010869565217391, Synset('organism.n.01'): 0.48777173913043476, Synset('benthos.n.02'): 0.2323369565217391}


## Analisi risultati

In [53]:
def evaluate_synset_frequency_vs_average(avg_freq, synset_freq_dict):
    print(f'Frequenza media: {avg_freq}')
    
    most_similar_syn_close_to_avg_freq = min(synset_freq_dict, key=lambda x: abs(synset_freq_dict[x] - avg_freq))
    freq_most_similar_syn_close_to_avg_freq = synset_freq_dict[most_similar_syn_close_to_avg_freq]
    print(f'Il synset con frequenza più vicina alla media è: {most_similar_syn_close_to_avg_freq} con frequenza {freq_most_similar_syn_close_to_avg_freq}')

    syn_max_than_avg_freq = {}
    syn_min_than_avg_freq = {}
    syn_close_to_avg_freq = {}
    deviation_threshold = 0.05
    
    for synset, freq in synset_freq_dict.items():
        if freq > avg_freq:
            syn_max_than_avg_freq[synset] = freq
        elif freq < avg_freq:
            syn_min_than_avg_freq[synset] = freq
        
        if abs(freq - avg_freq) < deviation_threshold:
            syn_close_to_avg_freq[synset] = freq
    
    top_synsets_above_avg = list(syn_max_than_avg_freq.keys())[:10]
    print("\nNumero di synset con frequenza MAGGIORE della media:", len(syn_max_than_avg_freq))
    print("I primi 10 synset che hanno frequenza MAGGIORE della media sono:")
    for synset in top_synsets_above_avg:
        print(synset)
    
    top_synsets_below_avg = list(syn_min_than_avg_freq.keys())[:10]
    print("\nNumero di synset con frequenza MINORE della media:", len(syn_min_than_avg_freq))
    print("I primi 10 synset che hanno frequenza MINORE della media sono:")
    for synset in top_synsets_below_avg:
        print(synset)
    
    top_synsets_close_avg = list(syn_close_to_avg_freq.keys())[:10]
    print("\nI primi 10 synset che hanno frequenza VICINA alla media sono:")
    for synset in top_synsets_close_avg:
        print(synset)
    
    return syn_max_than_avg_freq, syn_min_than_avg_freq, syn_close_to_avg_freq

### word_frequency()

In [54]:
syn_max_than_avg_wordfreq, syn_min_than_avg_wordfreq, syn_close_to_avg_wordfreq = evaluate_synset_frequency_vs_average(wordfreq_average_frequency, synset_wordfreq_dict)

Frequenza media: 1.7238083952626823e-05
Il synset con frequenza più vicina alla media è: Synset('finger-painting.n.02') con frequenza 1.72e-05

Numero di synset con frequenza MAGGIORE della media: 13698
I primi 10 synset che hanno frequenza MAGGIORE della media sono:
Synset('thing.n.12')
Synset('object.n.01')
Synset('whole.n.02')
Synset('living_thing.n.01')
Synset('parent.n.02')
Synset('life.n.10')
Synset('cell.n.02')
Synset('person.n.01')
Synset('animal.n.01')
Synset('plant.n.02')

Numero di synset con frequenza MINORE della media: 68417
I primi 10 synset che hanno frequenza MINORE della media sono:
Synset('entity.n.01')
Synset('physical_entity.n.01')
Synset('abstraction.n.06')
Synset('congener.n.03')
Synset('organism.n.01')
Synset('benthos.n.02')
Synset('dwarf.n.03')
Synset('heterotroph.n.01')
Synset('biont.n.01')
Synset('causal_agent.n.01')

I primi 10 synset che hanno frequenza VICINA alla media sono:
Synset('entity.n.01')
Synset('physical_entity.n.01')
Synset('abstraction.n.06')
S

### zipf_frequency()

In [55]:
syn_max_than_avg_wordfreq, syn_min_than_avg_wordfreq, syn_close_to_avg_wordfreq = evaluate_synset_frequency_vs_average(zipf_average_frequency, synset_zipf_freq_dict)

Frequenza media: 2.7441142300432566
Il synset con frequenza più vicina alla media è: Synset('wipeout.n.02') con frequenza 2.74

Numero di synset con frequenza MAGGIORE della media: 45102
I primi 10 synset che hanno frequenza MAGGIORE della media sono:
Synset('entity.n.01')
Synset('physical_entity.n.01')
Synset('abstraction.n.06')
Synset('thing.n.12')
Synset('object.n.01')
Synset('whole.n.02')
Synset('living_thing.n.01')
Synset('organism.n.01')
Synset('dwarf.n.03')
Synset('parent.n.02')

Numero di synset con frequenza MINORE della media: 37013
I primi 10 synset che hanno frequenza MINORE della media sono:
Synset('congener.n.03')
Synset('benthos.n.02')
Synset('heterotroph.n.01')
Synset('biont.n.01')
Synset('abdominoplasty.n.01')
Synset('agon.n.01')
Synset('beachhead.n.02')
Synset('cakewalk.n.02')
Synset('masterstroke.n.01')
Synset('res_gestae.n.02')

I primi 10 synset che hanno frequenza VICINA alla media sono:
Synset('entree.n.04')
Synset('disengagement.n.02')
Synset('wipeout.n.02')
Syn

Numero di parole che hanno frequenza maggiore o minore della media.

## Creazione del dizionario {synset: frequenza, frequenza_normalizzata} e del dataframe

In [56]:
def create_freq_dict_df_results(syn_freq_dict, norm_syn_freq_dict):
    synset_freq_normfreq_dict = {}
    for synset in syn_freq_dict:
        freq = syn_freq_dict[synset]
        norm_freq = norm_syn_freq_dict.get(synset, 0)
        synset_freq_normfreq_dict[synset] = {'Frequency': freq, 'Normalized Frequency': norm_freq}
    
    df_freqs = pd.DataFrame.from_dict(synset_freq_normfreq_dict, orient='index')
    df_freqs.reset_index(inplace=True)
    df_freqs.rename(columns={'index': 'Synset'}, inplace=True)
    
    return synset_freq_normfreq_dict, df_freqs

### word_frequency()

In [57]:
wordfreq_synset_freq_normfreq_dict, df_wordfreqs = create_freq_dict_df_results(synset_wordfreq_dict, norm_synset_wordfreq_dict)
primi_10_valori = {k: wordfreq_synset_freq_normfreq_dict[k] for k in list(wordfreq_synset_freq_normfreq_dict)[:10]}
print(primi_10_valori)
print(df_wordfreqs[:10])

{Synset('entity.n.01'): {'Frequency': 1.32e-05, 'Normalized Frequency': 0.7657463576738481}, Synset('physical_entity.n.01'): {'Frequency': 1.14e-05, 'Normalized Frequency': 0.6613263998092324}, Synset('abstraction.n.06'): {'Frequency': 2e-06, 'Normalized Frequency': 0.11602217540512849}, Synset('thing.n.12'): {'Frequency': 0.000525, 'Normalized Frequency': 1}, Synset('object.n.01'): {'Frequency': 3.63e-05, 'Normalized Frequency': 1}, Synset('whole.n.02'): {'Frequency': 0.000288, 'Normalized Frequency': 1}, Synset('congener.n.03'): {'Frequency': 4.68e-08, 'Normalized Frequency': 0.002714918904480007}, Synset('living_thing.n.01'): {'Frequency': 0.000145, 'Normalized Frequency': 1}, Synset('organism.n.01'): {'Frequency': 3.89e-06, 'Normalized Frequency': 0.22566313116297493}, Synset('benthos.n.02'): {'Frequency': 5.13e-08, 'Normalized Frequency': 0.002975968799141546}}
                           Synset     Frequency  Normalized Frequency
0           Synset('entity.n.01')  1.320000e-05    

### zipf_frequency()

In [58]:
zipfreq_synset_freq_normfreq_dict, df_zipfreqs = create_freq_dict_df_results(synset_zipf_freq_dict, zipf_norm_synset_freq_dict)
primi_10_valori = {k: zipfreq_synset_freq_normfreq_dict[k] for k in list(zipfreq_synset_freq_normfreq_dict)[:10]}
print(primi_10_valori)
print(df_zipfreqs[:10])

{Synset('entity.n.01'): {'Frequency': 4.12, 'Normalized Frequency': 0.5597826086956522}, Synset('physical_entity.n.01'): {'Frequency': 4.06, 'Normalized Frequency': 0.5516304347826086}, Synset('abstraction.n.06'): {'Frequency': 3.3, 'Normalized Frequency': 0.44836956521739124}, Synset('thing.n.12'): {'Frequency': 5.72, 'Normalized Frequency': 0.7771739130434782}, Synset('object.n.01'): {'Frequency': 4.56, 'Normalized Frequency': 0.6195652173913042}, Synset('whole.n.02'): {'Frequency': 5.46, 'Normalized Frequency': 0.7418478260869564}, Synset('congener.n.03'): {'Frequency': 1.67, 'Normalized Frequency': 0.22690217391304346}, Synset('living_thing.n.01'): {'Frequency': 5.16, 'Normalized Frequency': 0.7010869565217391}, Synset('organism.n.01'): {'Frequency': 3.59, 'Normalized Frequency': 0.48777173913043476}, Synset('benthos.n.02'): {'Frequency': 1.71, 'Normalized Frequency': 0.2323369565217391}}
                           Synset  Frequency  Normalized Frequency
0           Synset('entity.

Salvataggio su file

In [59]:
path_output = "../df/"
df_file_output = path_output + 'df_synset_wordfreq.csv'
df_wordfreqs.to_csv(df_file_output, index=False)

zipf

In [60]:
# path_output = "../df/"
# df_file_output = path_output + 'df_synset_wordfreq_zipf.csv'
# df_zipfreqs.to_csv(df_file_output, index=False)

## Risultati ottenuti

### Frequenza di un synset specifico

#### word_frequency()

In [61]:
synset = wordnet.synset('animal.n.01')
word_freq_dict = wordfreq_synset_freq_normfreq_dict.get(synset, 'Synset non classificato')
freq = word_freq_dict['Frequency']
freq_norm = word_freq_dict['Normalized Frequency']
print(f'{synset}: Frequency: {freq}, Normalized Frequency: {freq_norm}')

Synset('animal.n.01'): Frequency: 6.46e-05, Normalized Frequency: 1


#### zipf_frequency() 

In [62]:
synset = wordnet.synset('animal.n.01')
zipf_freq_dict = zipfreq_synset_freq_normfreq_dict.get(synset, 'Synset non classificato')
freq = zipf_freq_dict['Frequency']
freq_norm = zipf_freq_dict['Normalized Frequency']
print(f'{synset}: Frequency: {freq}, Normalized Frequency: {freq_norm}')

Synset('animal.n.01'): Frequency: 4.81, Normalized Frequency: 0.6535326086956521


### Synset con frequenza decrescente

#### word_frequency()

In [63]:
df_sorted_word_freqs = df_wordfreqs.sort_values(by='Frequency', ascending=False)
print(df_sorted_word_freqs.head(20))

                         Synset  Frequency  Normalized Frequency
36841          Synset('a.n.06')    0.02290                   1.0
29807          Synset('a.n.07')    0.02290                   1.0
36849          Synset('i.n.03')    0.01230                   1.0
72682        Synset('are.n.01')    0.00550                   1.0
73101         Synset('at.n.02')    0.00501                   1.0
36895         Synset('he.n.02')    0.00490                   1.0
32175        Synset('one.n.02')    0.00295                   1.0
73548        Synset('one.n.01')    0.00295                   1.0
15881        Synset('can.n.01')    0.00288                   1.0
39322        Synset('can.n.03')    0.00288                   1.0
73680        Synset('can.n.02')    0.00288                   1.0
32722       Synset('will.n.02')    0.00282                   1.0
35309       Synset('will.n.03')    0.00282                   1.0
32047       Synset('like.n.02')    0.00257                   1.0
32051       Synset('like.

#### zipf_frequency()

In [64]:
df_sorted_zipf_freqs = df_zipfreqs.sort_values(by='Frequency', ascending=False)
print(df_sorted_zipf_freqs.head(20))

                         Synset  Frequency  Normalized Frequency
36841          Synset('a.n.06')       7.36              1.000000
29807          Synset('a.n.07')       7.36              1.000000
36849          Synset('i.n.03')       7.09              0.963315
72682        Synset('are.n.01')       6.74              0.915761
73101         Synset('at.n.02')       6.70              0.910326
36895         Synset('he.n.02')       6.69              0.908967
32175        Synset('one.n.02')       6.47              0.879076
73548        Synset('one.n.01')       6.47              0.879076
73680        Synset('can.n.02')       6.46              0.877717
39322        Synset('can.n.03')       6.46              0.877717
15881        Synset('can.n.01')       6.46              0.877717
35309       Synset('will.n.03')       6.45              0.876359
32722       Synset('will.n.02')       6.45              0.876359
32051       Synset('like.n.01')       6.41              0.870924
32047       Synset('like.